In [1]:
import random
import numpy as np
import pandas as pd

from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn import preprocessing
from xgboost import XGBClassifier
from cleanlab import Datalab
import os
os.chdir('C:/Users/RH/A_whole_try_prog/ESM320_clean/data')

SEED = 100  # for reproducibility
np.random.seed(SEED)
random.seed(SEED)

In [2]:
#load and process the data
df = pd.read_csv("C:/Users/RH/A_whole_try_prog/ESM320_clean/data/ESM_dimension_320.csv")
row_to_delete = 1392
df = df[:row_to_delete+1]
df_c = df.copy()
df['label'] = preprocessing.LabelEncoder().fit_transform(df["label"])
df['label'] = df['label'].astype('category')
X_raw = df.drop(['No', 'Sequence', 'label'], axis=1)
labels = df["label"]


KeyError: 'label'

In [5]:
#cat_features = ["notes"]

#X_encoded = pd.get_dummies(X_raw, drop_first=True)

#numeric_features = ["exam_1", "exam_2", "exam_3"]
#scaler = StandardScaler()
#X_processed = X_encoded.copy()
#X_processed = scaler.fit_transform(X_encoded)


In [6]:
#choose one ML method as you want
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(penalty='l2',max_iter=10000)
model = clf

In [7]:
num_crossval_folds = 5
pred_probs = cross_val_predict(
    model,
    X_raw,
    labels,
    cv=num_crossval_folds,
    method="predict_proba",
)

In [8]:
KNN = NearestNeighbors(metric='euclidean')
KNN.fit(X_raw.values)

knn_graph = KNN.kneighbors_graph(mode="distance")

In [9]:
data = {"X": X_raw.values, "y": labels}
lab = Datalab(data, label_name="y")
lab.find_issues(pred_probs=pred_probs, knn_graph=knn_graph)

Finding label issues ...
Finding outlier issues ...
Finding near_duplicate issues ...
Finding non_iid issues ...
Error in non_iid: If a knn_graph is not provided, features must be provided to fit a new knn.
Failed to check for these issue types: [NonIIDIssueManager]

Audit complete. 233 issues found in the dataset.


In [10]:
lab.report()

Here is a summary of the different kinds of issues found in the data:

    issue_type  num_issues
         label         207
       outlier          26
near_duplicate           0

Dataset Information: num_examples: 1084, num_classes: 2


----------------------- label issues -----------------------

About this issue:
	Examples whose given label is estimated to be potentially incorrect
    (e.g. due to annotation error) are flagged as having label issues.
    

Number of examples with this issue: 207
Overall dataset quality in terms of this issue: 0.7989

Examples representing most severe instances of this issue:
     is_label_issue  label_score  given_label  predicted_label
846            True     0.019402            0                1
910            True     0.028643            0                1
750            True     0.039582            0                1
875            True     0.040382            0                1
882            True     0.040895            0                1


-

In [11]:
issue_results = lab.get_issues("label")
issue_results.head()

,is_label_issue,label_score,given_label,predicted_label
0,True,0.138675,1,0
1,True,0.254925,1,0
2,True,0.188532,1,0
3,True,0.472544,1,0
4,True,0.104556,1,0


In [12]:
outlier_results = lab.get_issues("outlier")

In [13]:
sorted_issues = issue_results.sort_values("label_score").index
X_raw_print = df_c.iloc[sorted_issues].assign(
    is_label_issue = issue_results["is_label_issue"].iloc[sorted_issues],
    label_score = issue_results["label_score"].iloc[sorted_issues],
    given_label=labels.iloc[sorted_issues],
    predicted_label=issue_results["predicted_label"].iloc[sorted_issues],
    outlier_label = outlier_results["is_outlier_issue"].iloc[sorted_issues],
    outlier_score = outlier_results["outlier_score"].iloc[sorted_issues]
)
X_raw_print.to_csv("6v3_clean.csv")
X_raw.iloc[sorted_issues].assign(
    given_label=labels.iloc[sorted_issues],
    predicted_label=issue_results["predicted_label"].iloc[sorted_issues]
).head()

,0,1,2,3,4,5,6,7,8,9,...,312,313,314,315,316,317,318,319,given_label,predicted_label
846,0.055671,-0.203267,0.209244,0.299846,-0.025067,-0.238533,-0.331110,-0.281764,-0.160348,-0.164183,...,-0.011503,0.283627,0.057359,0.254332,0.149765,0.033910,0.229669,0.075105,0,1
910,0.063344,-0.388523,0.330472,0.215353,-0.009694,-0.197672,-0.136367,-0.082993,-0.125764,-0.299276,...,0.147795,0.022309,0.094377,0.113881,-0.138335,0.053564,0.156247,-0.116531,0,1
750,0.105042,-0.137700,0.193816,0.355645,0.077005,-0.154625,-0.331690,-0.267747,-0.024495,-0.259550,...,0.032943,-0.051368,0.167843,-0.012312,0.108793,0.091843,0.283302,-0.083271,0,1
875,0.085311,-0.387707,0.300029,0.140690,-0.060954,-0.072838,-0.113678,0.001116,-0.009386,-0.119025,...,0.078416,0.161138,-0.007763,0.118178,-0.021098,0.128253,-0.097965,-0.220562,0,1
882,0.022685,-0.281447,0.349300,0.042933,-0.067725,-0.219354,-0.292148,-0.162812,-0.237684,-0.133125,...,0.161481,-0.005459,-0.022633,0.065071,0.029935,0.202144,0.168429,-0.128003,0,1


In [13]:
outlier_results = lab.get_issues("outlier")
sorted_outliers= outlier_results.sort_values("outlier_score").index

X_raw.iloc[sorted_outliers].head()

,exam_1,exam_2,exam_3,notes
3,0.61,0.94,0.78,NaN
7,100.00,100.00,1.00,NaN
0,53.00,77.00,9.00,3
4,48.00,90.00,9.00,1
8,0.00,56.00,96.00,"<p style=""font-size: 18px; color: #ff00ff; bac..."


In [14]:
duplicate_results = lab.get_issues("near_duplicate")
duplicate_results.sort_values("near_duplicate_score").head()

,is_near_duplicate_issue,near_duplicate_score,near_duplicate_sets,distance_to_nearest_neighbor
690,True,0.0,[246],0.0
185,True,0.0,[582],0.0
691,True,0.0,[294],0.0
168,True,0.0,[915],0.0
187,True,0.0,[27],0.0


In [15]:
X_raw.iloc[[690, 246]]

,exam_1,exam_2,exam_3,notes
690,78.0,58.0,86.0,great final presentation +10
246,78.0,58.0,86.0,great final presentation +10


In [20]:
X_raw.iloc[[185, 582]]

,exam_1,exam_2,exam_3,notes
185,90.0,67.0,77.0,missed class frequently -10
582,90.0,67.0,77.0,missed class frequently -10
